# OpenGridMap Notebook  -- Algorithm Inference

Before executing any code in this Notebook, you should 
1. Install what specified in Installation.ipynb
2. Understand DataPreparation.ipynb is also helpful

The goal of this notebook is to
1. Learn APIs of NetworkX
2. Inference and test with different algorithms

# Step1. Data Preparation

## Import necessary modules

In [6]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from networkx.algorithms import approximation as approx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
import math
import matplotlib.pyplot as plt
%matplotlib inline


def cleanMap(transJson, vornoiJson): 
    # Mark transformer
    tCenter = transJson['coordinates']
    tMarker =  Marker(location=tCenter)
    # Mark polygon
    tPoly = Polygon(locations=vornoiJson['coordinates'], weight=2,
                color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
                fill_color='#ccffcc') 
    
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=tCenter ,zoom=16)
    tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

def connect(broken):# Set connection to gis db
    if broken:
        conn.close()
    try:
        conn = psycopg2.connect("dbname=gis user=jennyzhou")
    except:
        print 'Fail to connect to Postgres Server'
    return conn

## Database connection

In [7]:
conn = connect(False)

## Select transformer, house within one vornoi area

In [8]:
cur = conn.cursor()
vid = 38
# Select vornoi polygon and transformer
cur.execute("SELECT ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(way, 4326)))), "
            "ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(tway, 4326)))) FROM vornoi WHERE vid = %d ;" % vid)
data = cur.fetchall()

# Select all the houses
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326)))) "
            "FROM vornoi_map where vid = %d and istransformer = false ;" % vid)
houses = cur.fetchall()
cur.close()

vornoiJson = json.loads(data[0][0])
transJson = json.loads(data[0][1])
houseJson = json.loads(houses[0][0])

# Step2. Data visualisation with ipyleaflet

In [336]:
tMap = cleanMap(transJson, vornoiJson)
tMap

In [200]:
# Mark houses
hmarkers = []
for hp in houseJson['coordinates'][0:1]:
    c = Circle(location= hp, weight=5, opacity = 0.5, color = '#00134d', radius = 2) 
    hmarkers.append(c)
    tMap.add_layer(c) 

In [198]:
for mk in hmarkers:
    tMap.remove_layer(mk)

### DB Usage
1. Most roads information can be obtained in table planet_osm_line
2. We use 'highway' field to filter different types of road.
3. Tags for 'highway':
   - Large road: 'residential'; Small road: 'footway'; Park road: 'service'; 'step' same as 'track'
   - The main tag we require by now is 'residential'


## Find roads near to a house within specified distance range

In [195]:
cur = conn.cursor()
### Main street from Garching
# cur.execute("SELECT ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(way, 4326))))) FROM planet_osm_line "
#             "WHERE ST_Within(way,(SELECT way FROM planet_osm_polygon WHERE osm_id = -30971 )) = true "
#             "AND highway in ('residential');")# in('primary','secondary','primary_link');")

cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(t1.way, 4326))), "
            "ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(t2.way, 4326))))) "
            "FROM (SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false LIMIT 2) AS t1, "
            "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
            "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "
            "WHERE ST_DWithin(t1.way, t2.way, 60) = true GROUP BY t1.way;" % (vid,vid))
hsRdMap = cur.fetchall()
tMap = cleanMap(transJson, vornoiJson)
cur.close()

# House point and nearest projected point on the road
for i in range(len(hsRdMap)):
    house = json.loads(hsRdMap[i][0])['coordinates']
    omk = Circle(location=house, weight=5, opacity = 0.5, color = '#00134d', radius = 2) 
    tMap.add_layer(omk)

    roadJson = json.loads(hsRdMap[i][1])
    for rd in roadJson['coordinates']:
        pl = Polyline(locations=rd, weight=2, color='red', opacity=0.8) 
        tMap.add_layer(pl)    
tMap


## Project houses to roads within specified distance range

In [201]:
cur = conn.cursor()
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(t1.way, 4326))), "
            "ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(ST_ClosestPoint(t2.way, t1.way), 4326)))) "
            "FROM (SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false LIMIT 2) AS t1, "
            "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
            "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "
            "WHERE ST_DWithin(t1.way, t2.way, 100) = true GROUP BY t1.way;" % (vid,vid))
hsRdMap = cur.fetchall()
tMap = cleanMap(transJson, vornoiJson)
# House point and nearest projected point on the road
for i in range(len(hsRdMap)):
    house = json.loads(hsRdMap[i][0])['coordinates']
    omk = Circle(location=house, weight=5, opacity = 0.5, color = '#00134d', radius = 2) 
    tMap.add_layer(omk)

    proJson = json.loads(hsRdMap[i][1])
    for proj in proJson['coordinates']:
        pmk = Circle(location=proj, weight=5, opacity = 0.5, color = '#003300', radius = 2) 
        tMap.add_layer(pmk)
        
        line = [house, proj]
        pl = Polyline(locations=line, weight=2, color='red', opacity=0.8) 
        tMap.add_layer(pl)  
        
# Method 2
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(t1.way, 4326))), "
            #"ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(t2.way, 4326))), "
            "ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_ClosestPoint(t2.way, t1.way), 4326))), "
            "ST_Distance(t1.way, t2.way) "
            "FROM "
            "(SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false LIMIT 2) AS t1 LEFT JOIN "
            "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
            "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "            
            "ON ST_DWithin(t1.way, t2.way, 100) = true;" % (vid,vid))
hsRdMap = cur.fetchall()
tMap = cleanMap(transJson, vornoiJson)
# House point and nearest projected point on the road
for i in range(len(hsRdMap)):
    house = json.loads(hsRdMap[i][0])['coordinates']
    omk = Circle(location=house, weight=5, opacity = 0.5, color = '#00134d', radius = 2) 
    tMap.add_layer(omk)
    
    proj = json.loads(hsRdMap[i][1])['coordinates']
    pmk = Circle(location=proj, weight=5, opacity = 0.5, color = '#003300', radius = 2) 
    tMap.add_layer(pmk)
    
    line = [house, proj]
    pl = Polyline(locations=line, weight=2, color='red', opacity=0.8) 
    tMap.add_layer(pl) 
cur.close()
tMap

## Project houses only to the nearest road

In [213]:
conn = connect(False)
# Select roads near one house
cur = conn.cursor()
# Create intermediate table
cur.execute("CREATE TABLE IF NOT EXISTS tempHsRd (hway geometry(Point,900913), rway geometry(LineString,900913), distance float);")
conn.commit()   
# clean entries
cur.execute("DELETE FROM temphsrd;")
conn.commit() 
cur.execute("INSERT INTO temphsrd "
            "(SELECT t1.way, t2.way, ST_Distance(t1.way, t2.way) "
            "FROM "
            "(SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false) AS t1 LEFT JOIN "
            "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
            "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "            
            "ON ST_DWithin(t1.way, t2.way, 100) = true);" % (vid,vid))
conn.commit() 

cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(hway, 4326))), "
            "ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_ClosestPoint(rway, hway), 4326))) "
            "FROM temphsrd "
            "WHERE (hway, distance) IN "
            "(SELECT hway, MIN(distance) FROM temphsrd GROUP BY hway);")
hsRdMap = cur.fetchall()


# cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(t3.hway, 4326))), "
#             "ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_ClosestPoint(t3.rway, t3.hway), 4326))) "
#             "FROM "
#             "(SELECT t1.way AS hway, t2.way AS rway, ST_Distance(t1.way, t2.way) AS distance "
#             "FROM "
#             "(SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false LIMIT 2) AS t1 LEFT JOIN "
#             "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
#             "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "            
#             "ON ST_DWithin(t1.way, t2.way, 100) = true) AS t3 LEFT JOIN "
#             "(SELECT t3.hway AS hway, MIN(t3.distance) AS max_distance FROM t3 GROUP BY t3.hway) AS t4 "
#             "ON t3.hway = t4.hway "
#             "AND t3.distance = t4.max_distance;" % (vid,vid))

# cur.execute("SELECT ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(t1.way, 4326)))), "
#             "ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(ST_ClosestPoint(t2.way, t1.way), 4326)))), "
#             "ST_Distance(t1.way, t2.way) "
#             "FROM (SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false LIMIT 3) AS t1, "
#             "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
#             "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "
#             "WHERE ST_DWithin(t1.way, t2.way, 100) = true GROUP BY t1.way;" % (vid,vid))

cur.close()


In [216]:
tMap = cleanMap(transJson, vornoiJson)

for i in range(len(hsRdMap)):
    house = json.loads(hsRdMap[i][0])['coordinates']
    omk = Circle(location=house, weight=5, opacity = 0.5, color = '#00134d', radius = 2) 
    tMap.add_layer(omk)

    proj = json.loads(hsRdMap[i][1])['coordinates']
    pmk = Circle(location=proj, weight=5, opacity = 0.5, color = '#003300', radius = 2) 
    tMap.add_layer(pmk)

    line = [house, proj]
    pl = Polyline(locations=line, weight=2, color='red', opacity=0.8) 
    tMap.add_layer(pl)    
tMap

# Step3. Construct network graph

## Preparing vertices and edges

In [5]:
cur = conn.cursor()
# Select vornoi polygon and transformer
cur.execute("SELECT oid FROM vornoi_map WHERE vid = %d and istransformer = True;"%vid)
tid = cur.fetchall()[0][0]
cur.execute("SELECT t1.oid, t2.oid, ST_Distance(t1.way, t2.way) "
            "FROM (SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t1, "
            "(SELECT oid, way FROM vornoi_map WHERE vid = %d and istransformer = False) AS t2 "
            "WHERE t1.oid != t2.oid ;"% (vid,vid))
nodes = cur.fetchall()
cur.close()

In [79]:
def saveGraph(conn, vid, edgeList, name):
    # Store result into database
    cur = conn.cursor()
    # Check wheter already stored
    cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326)))"
                "FROM graph WHERE type = '%s' AND vid = %d;" %(name,vid))
    lines = cur.fetchall()
    # If not, store minimum spanning tree into database
    if len(lines)==0:
        print 'Storing topology of %s into db'%name
        cur.execute("CREATE TABLE IF NOT EXISTS templines (way geometry(LineString,900913));")
        conn.commit()   
        # clean entries
        cur.execute("DELETE FROM templines;")
        conn.commit()   

        for edge in edgeList:
            (start, end) = edge
            cur.execute("INSERT INTO templines (SELECT ST_MakeLine(t1.way, t2.way) "
                        "FROM (SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t1, "
                        "(SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t2 "
                        "WHERE t1.oid = %d AND t2.oid = %d);"% (vid, vid, start, end ))
            conn.commit()   

        cur.execute("INSERT INTO graph (vid, type, way) SELECT %d, '%s', ST_Collect(way) FROM templines;"%(vid, name))
        conn.commit()
        cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326)))"
                    "FROM graph WHERE type = '%s' AND vid = %d;" %(name,vid))
        lines = cur.fetchall()
    cur.close()
    return lines

## Network Simplex 

In [80]:
# Build  directed graph
graph = []
for ele in nodes:
    w = int(math.floor(ele[2]))
    graph.append("%d %d %d" % (ele[0],ele[1],w))
G_ = nx.parse_edgelist(graph, create_using = nx.DiGraph(), nodetype = int, data=(('weight',int),))
assert G_.has_node(tid)

for n in G_.nodes(data=True):
    n[1]['demand'] = 1
G_.add_node(tid, demand = -len(G_.nodes())+1)
flowCost, flowDict = nx.capacity_scaling(G_)
#flowCost, flowDict = nx.network_simplex(G_)
NS = sorted([(u, v) for u in flowDict for v in flowDict[u] if flowDict[u][v] > 0])

print approx.node_connectivity(G_), len(NS)
lines = saveGraph(conn, vid, NS, 'NS')

90 90
Storing topology of NS into db


## Minimum Spanning Tree

In [9]:
graph = []
for ele in nodes:
    graph.append("%d %d %f" % ele)
G = nx.parse_edgelist(graph, nodetype = int, data=(('weight',float),))

MST = nx.minimum_spanning_tree(G)
print approx.node_connectivity(G), len(MST.nodes())
#lines = saveGraph(conn, vid, MST.edges(), 'MST')

90 91


## Minimum Spanning Arborescence 

In [84]:
graph = []
for ele in nodes:
    graph.append("%d %d %f" % ele)
G_ = nx.parse_edgelist(graph, create_using = nx.DiGraph(), nodetype = int, data=(('weight',float),))
assert G_.has_node(tid)

MSA = nx.minimum_spanning_arborescence(G_)
print approx.node_connectivity(G_), len(MSA.nodes())

lines = saveGraph(conn, vid, MSA.edges(), 'MSA')

90 91
Storing topology of MSA into db


## Minimum Branching 

In [89]:
# Not applicable
# graph = []
# for ele in nodes:
#     graph.append("%d %d %f" % ele)
# G_ = nx.parse_edgelist(graph, create_using = nx.DiGraph(), nodetype = int, data=(('weight',float),))
# assert G_.has_node(tid)

# MB = nx.minimum_branching(G_)
# print approx.node_connectivity(G_),len(MB.nodes())
# lines = saveGraph(conn, vid, MB.edges(), 'MB')
# print len(res[0][0])


# Step4. Display graph on map

In [87]:
tmpJson = json.loads(lines[0][0])
tmpMap = cleanMap(transJson, vornoiJson)
for l in tmpJson['coordinates']:
    pl = Polyline(locations=l, weight=2, color='#2929a3', opacity=0.5)
    tmpMap.add_layer(pl)
tmpMap